In [9]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# 패키지 import
import datetime as dt
import pandas as pd
import requests
import time
import urllib.request #
import re 

In [11]:
from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [131]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Selenium을 사용하여 웹 드라이버를 시작합니다 (Chrome 브라우저를 예시로 사용합니다)
driver = webdriver.Chrome()

# 웹 페이지로 이동합니다
driver.get('https://comic.naver.com/webtoon/weekday')  # 실제 웹 페이지 URL을 넣어주세요

# 페이지가 로드될 때까지 기다립니다 (필요에 따라 시간을 조정하세요)
driver.implicitly_wait(10)  # 예: 10초간 기다립니다

# 이차원 리스트를 초기화합니다
results = [[] for _ in range(7)]  # 7개의 리스트를 가진 이차원 리스트 생성

# div:nth-child(1)부터 div:nth-child(7)까지의 선택자를 동적으로 처리합니다
for i in range(1, 8):  # 1부터 7까지 반복합니다
    selector = f'#container > div.component_wrap.type2 > div.WeekdayMainView__daily_all_wrap--UvRFc > div:nth-child({i}) > ul > li > div > a > span > span'
    elements = driver.find_elements(By.CSS_SELECTOR, selector)
    for element in elements:
        text = element.text.strip()
        results[i - 1].append(text)  # i-1 인덱스에 결과 추가 (0부터 시작하도록)

# 작업을 마치고 나면 드라이버를 종료합니다
driver.quit()

In [136]:
# 요일별 웹툰 개수
lengths = [len(row) for row in results]
lengths

[94, 96, 92, 93, 91, 102, 100]

In [155]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

def naver_webtoon_scraper():
    
    url = 'https://comic.naver.com/webtoon'
    
    link_list = []
    day_list = []  # 요일
    rank_list = []  # 순위
    title_list = []  # 제목
    rating_list = []  # 별점
    
    story_list = []  # 스토리
    keywords_list = []  # 키워드
    like_list = [] # 관심
    num_list = [] # 몇화까지
    age_list = [] # 이용가
    
    inform_list = []
    
    driver = webdriver.Chrome()
    
    # 미리 로그인 (성인웹툰 접근용 )
    driver.get('http://nid.naver.com/nidlogin.login')
    time.sleep(20)
    
    
    for index, day in enumerate(['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']):
        day_url = f'{url}?tab={day}'
        driver.get(day_url)
        driver.implicitly_wait(10)      
        
        # 각 요일별 웹툰 개수 가져오기
        n = lengths[index]
        for i in range(1, n+1):
            
            # 각 웹툰의 링크 가져오기
            link_selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > a'
            link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
            webtoon_link = link_element.get_attribute('href')
            link_list.append(webtoon_link)
            
            # 웹툰이름, 작가, 별점을 가져올 수 있음. -> information에 들어있다.
            selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > div'
            element = driver.find_element(By.CSS_SELECTOR, selector)
            information = element.text.strip()
            print(information)
            inform_list.append(information)
            #information2 = information.split('\n')  #### 제목이 길면 오류가 나서 임시로 주석처리
            #print(information2)
            
            ## information을 약간의 전처리 한게 information2였음
            
            rank_list.append(i)
            
#             title = information2[0]
            
#             if title in title_list: # 주 2회 이상 연재시
#                 day_list[title_list.index(title)] += ', ' + day
#                 rank_list[title_list.index(title)] = str(rank_list[title_list.index(title)]) + ', ' + str(i) 
#                 continue

            day_list.append(day)
            # title_list.append(title)
            # rating_list.append(information2[-1])
        
        
            # 상세 페이지로 이동
            driver.get(webtoon_link)
            driver.implicitly_wait(10)
            
            html = driver.page_source
            soup = bs(html, 'html.parser')
            
            ### 장르, 요약, 키워드, 관심, 몇화
            xpath_episode_info = '//*[@id="content"]/div[1]/div/div[2]/p'
            xpath_keywords = '//*[@id="content"]/div[1]/div/div[2]/div/div'
            xpath_interest = '//*[@id="content"]/div[2]/div/button[1]/span[2]'
            
            # 연령대 오류가 뜸 --> 이유 못 찾음!!!
            #xpath_age = '//*[@id="content"]/div[1]/div/div[1]/em/text()[2]'
            
            episode_info = driver.find_element(By.XPATH, xpath_episode_info).text
            keywords = driver.find_element(By.XPATH, xpath_keywords).text
            interest = driver.find_element(By.XPATH, xpath_interest).text
           
            #age = driver.find_element(By.XPATH, xpath_age).text
            
            
            # 몇화까지 나왔는지 보는 부분
            # -> 가끔씩 다음 화는 ㅇㅇ에 올라옵니다 와 같은 블럭이 생기면 다른걸 넣어줘야함
            # 근데 오류뜸.. 이유 아직 모름
#             try:
#                 xpath_total_episodes = '//*[@id="content"]/div[3]/div[1]/div[1]'
#                 total_episodes = driver.find_element(By.XPATH, xpath_total_episodes).text
#             except NoSuchElementException:
#                 # 첫 번째 XPath가 없는 경우 대체할 XPath 사용
#                 xpath_total_episodes = '//*[@id="content"]/div[3]/div[2]/div[1]'
#                 total_episodes = driver.find_element(By.XPATH, xpath_total_episodes).text
            
            story_list.append(episode_info)  # 스토리
            keywords_list.append(keywords) # 키워드
            like_list.append(interest) # 관심
            # num_list.append(total_episodes) # 몇화까지
            #age_list.append(age) # 이용가
            
            driver.back()
            driver.implicitly_wait(10)
            
    driver.quit()
    
    data = {
    'Title': title_list,
    'Link': link_list,
    'Day': day_list,
    'Rank': rank_list,
    'Rating': rating_list,
    'Story': story_list,
    'Keywords': keywords_list,
    'Likes': like_list,
    'Num': num_list,
    #'Age': age_list
}
    
    return pd.DataFrame(data)
    
# 함수 호출
df = naver_webtoon_scraper()


뷰티풀 군바리
설이 / 윤성원
별점
9.80
참교육
채용택 / 한가람
별점
9.88
똑 닮은 딸
이담
별점
9.98
백수세끼
치즈
별점
9.86
왕과의 야행
보리
별점
9.98
윈드브레이커
조용석
별점
9.84
귀촌리
황양 / 이대한
별점
9.95
신의 탑
SIU
별점
9.86
퀘스트지상주의
박만사, 유누니 / 박만사, 태완
별점
9.63
소녀의 세계
모랑지
별점
9.92
신화급 귀속 아이템을 손에 넣었다
정선율 / 헤스
별점
9.91



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[1]/div/div[2]/p"}
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C4DAEEA2+31554]
	(No symbol) [0x00007FF6C4D27ED9]
	(No symbol) [0x00007FF6C4BE872A]
	(No symbol) [0x00007FF6C4C38434]
	(No symbol) [0x00007FF6C4C3853C]
	(No symbol) [0x00007FF6C4C7F6A7]
	(No symbol) [0x00007FF6C4C5D06F]
	(No symbol) [0x00007FF6C4C7C977]
	(No symbol) [0x00007FF6C4C5CDD3]
	(No symbol) [0x00007FF6C4C2A33B]
	(No symbol) [0x00007FF6C4C2AED1]
	GetHandleVerifier [0x00007FF6C50B8B1D+3217341]
	GetHandleVerifier [0x00007FF6C5105AE3+3532675]
	GetHandleVerifier [0x00007FF6C50FB0E0+3489152]
	GetHandleVerifier [0x00007FF6C4E5E776+750614]
	(No symbol) [0x00007FF6C4D3375F]
	(No symbol) [0x00007FF6C4D2EB14]
	(No symbo

In [151]:
## 현재 문제 1) 연령가가 크롤링 X
## 2) 총 몇화인지 잘 안나옴
## 3) 제목이 길면 안넘어가진다.
## 4) 설명이 너무 길면 멈춤.


### 더보기로 키워드가 늘어나는 경우가 있음................